<a href="https://colab.research.google.com/github/MartinMashalov/ANNSource/blob/master/VertaforeEndpoints.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Baseline Scipt

In [ ]:
!pip install zeep

In [ ]:
import requests
import xml.etree.ElementTree as ET
import json

def extract_data(element: ET.Element) -> dict:
    data = {}
    for child in element:
        tag = child.tag.split('}')[-1]
        if len(child) == 0:
            data[tag] = child.text
        else:
            for i, subchild in enumerate(child):
                subtag = subchild.tag.split('}')[-1]
                for subsubchild in subchild:
                    subsubtag = subsubchild.tag.split('}')[-1]
                    data[f"{tag}_{i+1}_{subtag}_{subsubtag}"] = subsubchild.text
    return data

# Define your credentials
agency_no = "1002683-1"
login_id = "Bigbridgeai"
password = "Welcome1234"
employee_code = "!(-"

# AMS360 SOAP API endpoint
url = "https://affwsapi.ams360.com/v2/service.asmx"

# SOAPAction header
headers = {
    "Content-Type": "text/xml; charset=utf-8",
    "SOAPAction": "http://www.WSAPI.AMS360.com/v2.0/Authenticate"
}

# SOAP envelope with authentication details
soap_body = f"""<?xml version="1.0" encoding="utf-8"?>
<soap:Envelope xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:xsd="http://www.w3.org/2001/XMLSchema" xmlns:soap="http://schemas.xmlsoap.org/soap/envelope/">
  <soap:Body>
    <AgencyNo xmlns="http://www.WSAPI.AMS360.com/v2.0">{agency_no}</AgencyNo>
    <LoginId xmlns="http://www.WSAPI.AMS360.com/v2.0">{login_id}</LoginId>
    <Password xmlns="http://www.WSAPI.AMS360.com/v2.0">{password}</Password>
    <EmployeeCode xmlns="http://www.WSAPI.AMS360.com/v2.0">{employee_code}</EmployeeCode>
  </soap:Body>
</soap:Envelope>"""

# Send the POST request
response = requests.post(url, data=soap_body, headers=headers)

# Check if the request was successful
if response.status_code == 200:
    print("Authentication successful.")

    # Parse the XML response
    root = ET.fromstring(response.text)

    # Define namespaces
    namespaces = {
        'soap': 'http://schemas.xmlsoap.org/soap/envelope/',
        'wsapi': 'http://www.WSAPI.AMS360.com/v2.0'
    }

    # Use XPath to find the token
    token = root.find('.//soap:Header/wsapi:WSAPIAuthToken/wsapi:Token', namespaces)

    if token is not None:
        ticket = token.text
        print("Token:", token.text)
    else:
        print("Token not found.")
else:
    print(f"Authentication failed with status code: {response.status_code}")
    print("Response:")
    print(response.text)


In [ ]:
import requests
import xml.etree.ElementTree as ET
import csv
from typing import List, Dict
from pprint import pprint

# getting customer and policy details

BASE_URL = "https://wsapi.ams360.com/v3/WSAPIService.svc"

def send_soap_request(session_token: str, action: str, body: str) -> ET.Element:
    soap_envelope = f"""<?xml version="1.0" encoding="utf-8"?>
<s:Envelope xmlns:s="http://schemas.xmlsoap.org/soap/envelope/">
    <s:Header>
        <h:WSAPISession xmlns:h="http://www.WSAPI.AMS360.com/v3.0" xmlns:i="http://www.w3.org/2001/XMLSchema-instance">
            <h:Ticket>{session_token}</h:Ticket>
        </h:WSAPISession>
    </s:Header>
    <s:Body>
        {body}
    </s:Body>
</s:Envelope>
    """

    headers = {
        'Content-Type': 'text/xml; charset=utf-8',
        'SOAPAction': f'http://www.WSAPI.AMS360.com/v3.0/WSAPIServiceContract/{action}'
    }
    print(soap_envelope)

    response = requests.post(BASE_URL, data=soap_envelope, headers=headers)
    print(response.content)
    return ET.fromstring(response.content)

def extract_data(element: ET.Element) -> dict:
    data = {}
    for child in element:
        tag = child.tag.split('}')[-1]
        if len(child) == 0:
            data[tag] = child.text
        else:
            for i, subchild in enumerate(child):
                subtag = subchild.tag.split('}')[-1]
                for subsubchild in subchild:
                    subsubtag = subsubchild.tag.split('}')[-1]
                    data[f"{tag}_{i+1}_{subtag}_{subsubtag}"] = subsubchild.text
    return data

def get_customer_list(session_token: str, name_prefix: str = '%') -> List[Dict[str, str]]:
    body = f"""
<CustomerGetListByNamePrefix xmlns="http://www.WSAPI.AMS360.com/v3.0">
    <Request xmlns:d4p1="http://www.WSAPI.AMS360.com/v3.0/DataContract" xmlns:i="http://www.w3.org/2001/XMLSchema-instance">
        <d4p1:NamePrefix>{name_prefix}</d4p1:NamePrefix>
        <d4p1:CustomerType></d4p1:CustomerType>
        <d4p1:IsBrokersCustomer>false</d4p1:IsBrokersCustomer>
    </Request>
</CustomerGetListByNamePrefix>
    """
    root = send_soap_request(session_token, "CustomerGetListByNamePrefix", body)
    customers = root.findall(".//{http://www.WSAPI.AMS360.com/v3.0/DataContract}CustomerInfo")
    return [extract_data(customer) for customer in customers]

def get_vendor_list(session_token: str) -> List[Dict[str, str]]:
    body = f"""
  <VendorGetListAll xmlns="http://www.WSAPI.AMS360.com/v3.0">
    <Request xmlns:d4p1="http://www.WSAPI.AMS360.com/v3.0/DataContract" xmlns:i="http://www.w3.org/2001/XMLSchema-instance">
    </Request>
  </VendorGetListAll>
    """
    root = send_soap_request(session_token, "VendorGetListAll", body)
    customers = root.findall(".//{http://www.WSAPI.AMS360.com/v3.0/DataContract}CustomerInfo")
    return [extract_data(customer) for customer in customers]

def get_policies_by_customer_number(session_token: str, customer_number: str) -> List[Dict[str, str]]:
    body = f"""
<PolicyGetListByCustomerNumber xmlns="http://www.WSAPI.AMS360.com/v3.0">
    <Request xmlns:d4p1="http://www.WSAPI.AMS360.com/v3.0/DataContract" xmlns:i="http://www.w3.org/2001/XMLSchema-instance">
        <d4p1:CustomerNumber>{customer_number}</d4p1:CustomerNumber>
    </Request>
</PolicyGetListByCustomerNumber>
    """
    root = send_soap_request(session_token, "PolicyGetListByCustomerNumber", body)
    policies = root.findall(".//{http://www.WSAPI.AMS360.com/v3.0/DataContract}PolicyInfo")
    return [extract_data(policy) for policy in policies]

def get_cliams_by_customer_id(session_token: str, customer_id: str):
    body = f"""
  <ClaimGetListByCustomerId xmlns="http://www.WSAPI.AMS360.com/v3.0">
      <Request xmlns:d4p1="http://www.WSAPI.AMS360.com/v3.0/DataContract" xmlns:i="http://www.w3.org/2001/XMLSchema-instance">
          <d4p1:CustomerId>{customer_id}</d4p1:CustomerId>
      </Request>
  </ClaimGetListByCustomerId>
      """
    root = send_soap_request(session_token, "PolicyGetListByCustomerNumber", body)
    claims = root.findall(".//{http://www.WSAPI.AMS360.com/v3.0/DataContract}PolicyInfo")
    return [extract_data(claim) for claim in claims]

def get_policy_details(session_token: str, policy_id: str) -> Dict[str, str]:
    body = f"""
<PolicyGet xmlns="http://www.WSAPI.AMS360.com/v3.0">
    <Request xmlns:d4p1="http://www.WSAPI.AMS360.com/v3.0/DataContract" xmlns:i="http://www.w3.org/2001/XMLSchema-instance">
        <d4p1:PolicyId>{policy_id}</d4p1:PolicyId>
    </Request>
</PolicyGet>
    """
    root = send_soap_request(session_token, "PolicyGet", body)
    policy = root.find(".//{http://www.WSAPI.AMS360.com/v3.0/DataContract}Policy")
    return extract_data(policy) if policy is not None else {}

def write_to_csv(data: List[Dict[str, str]], filename: str):
    if not data:
        print(f"No data to write to {filename}")
        return

    keys = ['CustomerNumber'] + list(set().union(*(d.keys() for d in data)))
    with open(filename, 'w', newline='', encoding='utf-8') as output_file:
        dict_writer = csv.DictWriter(output_file, keys)
        dict_writer.writeheader()
        dict_writer.writerows(data)

def main(session_token: str):
    customers_and_policies = []
    detailed_policies = []
    customers = get_customer_list(session_token)
    print(f"Found {len(customers)} customers")

    for customer in customers:
        customer_number = customer.get('CustomerNumber')
        if customer_number:
            policies = get_policies_by_customer_number(session_token, customer_number)
            print(f"Found {len(policies)} policies for customer {customer_number}")

            if policies:
                for policy in policies:
                    customers_and_policies.append({**customer, **policy})
                    policy_id = policy.get('PolicyId')
                    if policy_id:
                        policy_details = get_policy_details(session_token, policy_id)
                        policy_details['CustomerNumber'] = customer_number
                        detailed_policies.append(policy_details)
            else:
                customers_and_policies.append(customer)

    write_to_csv(customers_and_policies, 'vertafore_customers_and_policies.csv')
    print(f"Data written to vertafore_customers_and_policies.csv")

    write_to_csv(detailed_policies, 'vertafore_detailed_policies.csv')
    print(f"Detailed policy data written to vertafore_detailed_policies.csv")


# Use the session token obtained from the login process
#main(ticket)

# define a specific customer ID
#spec_cust_number: str = '14694'
customer_arr = get_customer_list(ticket)
#policy_arr = get_policies_by_customer_number(ticket, spec_cust_number)
#get_policy_details(ticket, 'f4735ede-fcb4-4f2e-ae98-1c75872fbc4a')
#get_vendor_list(ticket)

#customer_arr = get_customer_list(ticket)

In [ ]:
# implement the policy transaction endpoint
import requests

# Define the token and other request parameters
#token = "your_auth_token"  # Replace with the actual token
policy_id = "2e0ab7ac-c116-46d6-8ef9-08c2f0a6f20b"  # Replace with the actual Policy ID
transaction_effective_date = "2023-01-01"  # Replace with the desired date
get_related_data = "true"  # Use "true" or "false" as needed

# AMS360 SOAP API endpoint
url = "https://affwsapi.ams360.com/v2/service.asmx"

# SOAPAction header
headers = {
    "Content-Type": "text/xml; charset=utf-8",
    "SOAPAction": "http://www.WSAPI.AMS360.com/v2.0/GetPPAPolicyDetail"
}

# SOAP envelope with request details
soap_body = f"""<?xml version="1.0" encoding="utf-8"?>
<soap:Envelope xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:xsd="http://www.w3.org/2001/XMLSchema" xmlns:soap="http://schemas.xmlsoap.org/soap/envelope/">
  <soap:Header>
    <WSAPIAuthToken xmlns="http://www.WSAPI.AMS360.com/v2.0">
      <Token>{ticket}</Token>
    </WSAPIAuthToken>
  </soap:Header>
  <soap:Body>
    <GetPPAPolicyDetail_Request xmlns="http://www.WSAPI.AMS360.com/v2.0">
      <PolicyId>{policy_id}</PolicyId>
      <GetRelatedData>true</GetRelatedData>
    </GetPPAPolicyDetail_Request>
  </soap:Body>
</soap:Envelope>"""

# Send the POST request
response = requests.post(url, data=soap_body, headers=headers)
print(response.content)

# Check if the request was successful
if response.status_code == 200:
    print("Request successful.")
    print("Response:")
    print(response.text)
else:
    print(f"Request failed with status code: {response.status_code}")
    print("Response:")
    print(response.text)


In [ ]:
import xml.etree.ElementTree as ET
import json

# Parse the XML
root = ET.fromstring(response.content)

# Helper function to convert XML element to dictionary with lists
def xml_to_dict(element):
    # If element has no children, return text directly
    if len(element) == 0:
        return element.text

    result = {}
    for child in element:
        tag = child.tag.split("}")[1]  # Remove namespace

        # Handle repeated elements (multiple occurrences of the same tag)
        if tag in result:
            # Ensure the existing entry is a list, then append the new entry
            if not isinstance(result[tag], list):
                result[tag] = [result[tag]]
            result[tag].append(xml_to_dict(child))
        else:
            result[tag] = xml_to_dict(child)

    return result


# Extract the main response from within the SOAP envelope and body
namespaces = {'soap': 'http://schemas.xmlsoap.org/soap/envelope/'}
body = root.find('.//soap:Body', namespaces)
response_data = xml_to_dict(body)

# Convert the dictionary to JSON
json_data = json.dumps(response_data, indent=2)

# Display the JSON output
print(json_data)


In [ ]:
customer_number, customer_id = customer_arr[0]['CustomerNumber'], customer_arr[0]['CustomerId']
policy_arr = get_policies_by_customer_number(ticket, customer_number)
policy_id, policy_number = policy_arr[0]['PolicyId'], policy_arr[0]['PolicyNumber']
policy_details = get_policy_details(ticket, policy_id)
policy_details

## Invoices

In [ ]:
policy_id = policy_arr[2]['PolicyId']

In [ ]:
# get invoices


# implement the policy transaction endpoint
import requests

# Define the token and other request parameters
#token = "your_auth_token"  # Replace with the actual token
#policy_id = "2e0ab7ac-c116-46d6-8ef9-08c2f0a6f20b"  # Replace with the actual Policy ID
transaction_effective_date = "2023-01-01"  # Replace with the desired date
get_related_data = "true"  # Use "true" or "false" as needed

# AMS360 SOAP API endpoint
url = "https://affwsapi.ams360.com/v2/service.asmx"

# SOAPAction header
headers = {
    "Content-Type": "text/xml; charset=utf-8",
    "SOAPAction": "http://www.WSAPI.AMS360.com/v2.0/PostInvoice"
}

# SOAP envelope with request details
soap_body = f"""<?xml version="1.0" encoding="utf-8"?>
<soap:Envelope xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:xsd="http://www.w3.org/2001/XMLSchema" xmlns:soap="http://schemas.xmlsoap.org/soap/envelope/">
  <soap:Header>
    <WSAPIAuthToken xmlns="http://www.WSAPI.AMS360.com/v2.0">
      <Token>{ticket}</Token>
    </WSAPIAuthToken>
  </soap:Header>
  <soap:Body>
    <PostInvoice_Request xmlns="http://www.WSAPI.AMS360.com/v2.0">
      <PolicyId>{policy_id}</PolicyId>
    </PostInvoice_Request>
  </soap:Body>
</soap:Envelope>"""

# Send the POST request
response = requests.post(url, data=soap_body, headers=headers)
print(response.content)


# Parse the XML
root = ET.fromstring(response.content)

# Helper function to convert XML element to dictionary
def xml_to_dict(element):
    if len(element) == 0:  # If no children, return text
        return element.text
    result = {}
    for child in element:
        tag = child.tag.split("}")[-1]  # Remove namespace
        if tag in result:
            if not isinstance(result[tag], list):
                result[tag] = [result[tag]]
            result[tag].append(xml_to_dict(child))
        else:
            result[tag] = xml_to_dict(child)
    return result

# Extract relevant data from the SOAP body
namespaces = {'soap': 'http://schemas.xmlsoap.org/soap/envelope/'}
body = root.find('.//soap:Body', namespaces)
response_data = xml_to_dict(body)

# Convert the dictionary to JSON
json_data = json.dumps(response_data, indent=2)

# Display the JSON output
print(json_data)

# Check if the request was successful
if response.status_code == 200:
    print("Request successful.")
    print("Response:")
    print(response.text)
else:
    print(f"Request failed with status code: {response.status_code}")
    print("Response:")
    print(response.text)

## brokers

In [ ]:
# get brokers


# implement the policy transaction endpoint
import requests

# broker prefix
broker_prefix: str = ' '

# AMS360 SOAP API endpoint
url = "https://affwsapi.ams360.com/v2/service.asmx"

# SOAPAction header
headers = {
    "Content-Type": "text/xml; charset=utf-8",
    "SOAPAction": "http://www.WSAPI.AMS360.com/v2.0/GetBrokerList"
}

# SOAP envelope with request details
soap_body = f"""<?xml version="1.0" encoding="utf-8"?>
<soap:Envelope xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:xsd="http://www.w3.org/2001/XMLSchema" xmlns:soap="http://schemas.xmlsoap.org/soap/envelope/">
  <soap:Header>
    <WSAPIAuthToken xmlns="http://www.WSAPI.AMS360.com/v2.0">
      <Token>{ticket}</Token>
    </WSAPIAuthToken>
  </soap:Header>
  <soap:Body>
    <GetBrokerList_Request xmlns="http://www.WSAPI.AMS360.com/v2.0">
      <LastNamePrefix>{broker_prefix}</LastNamePrefix>
      <FilterActive>true</FilterActive>
    </GetBrokerList_Request>
  </soap:Body>
</soap:Envelope>"""

# Send the POST request
response = requests.post(url, data=soap_body, headers=headers)
print(response.content)

# Parse the XML
root = ET.fromstring(response.content)

# Helper function to convert XML element to dictionary
def xml_to_dict(element):
    if len(element) == 0:  # If no children, return text
        return element.text
    result = {}
    for child in element:
        tag = child.tag.split("}")[-1]  # Remove namespace
        if tag in result:
            if not isinstance(result[tag], list):
                result[tag] = [result[tag]]
            result[tag].append(xml_to_dict(child))
        else:
            result[tag] = xml_to_dict(child)
    return result

# Extract relevant data from the SOAP body
namespaces = {'soap': 'http://schemas.xmlsoap.org/soap/envelope/'}
body = root.find('.//soap:Body', namespaces)
response_data = xml_to_dict(body)

# Convert the dictionary to JSON
json_data = json.dumps(response_data, indent=2)

# Display the JSON output
print(json_data)

# Check if the request was successful
if response.status_code == 200:
    print("Request successful.")
    print("Response:")
    print(response.text)
else:
    print(f"Request failed with status code: {response.status_code}")
    print("Response:")
    print(response.text)

## claims

## emoloyees

In [ ]:
# get employees


# implement the policy transaction endpoint
import requests

# broker prefix
employee_prefix: str = '%'

# AMS360 SOAP API endpoint
url = "https://affwsapi.ams360.com/v2/service.asmx"

# SOAPAction header
headers = {
    "Content-Type": "text/xml; charset=utf-8",
    "SOAPAction": "http://www.WSAPI.AMS360.com/v2.0/GetEmployeeList"
}

# SOAP envelope with request details
soap_body = f"""<?xml version="1.0" encoding="utf-8"?>
<soap:Envelope xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:xsd="http://www.w3.org/2001/XMLSchema" xmlns:soap="http://schemas.xmlsoap.org/soap/envelope/">
  <soap:Header>
    <WSAPIAuthToken xmlns="http://www.WSAPI.AMS360.com/v2.0">
      <Token>{ticket}</Token>
    </WSAPIAuthToken>
  </soap:Header>
  <soap:Body>
    <GetEmployeeList_Request xmlns="http://www.WSAPI.AMS360.com/v2.0">
      <LastNamePrefix>{employee_prefix}</LastNamePrefix>
      <EmpType></EmpType>
    </GetEmployeeList_Request>
  </soap:Body>
</soap:Envelope>"""

# Send the POST request
response = requests.post(url, data=soap_body, headers=headers)
print(response.content)

# Parse the XML
root = ET.fromstring(response.content)

# Helper function to convert XML element to dictionary
def xml_to_dict(element):
    if len(element) == 0:  # If no children, return text
        return element.text
    result = {}
    for child in element:
        tag = child.tag.split("}")[-1]  # Remove namespace
        if tag in result:
            if not isinstance(result[tag], list):
                result[tag] = [result[tag]]
            result[tag].append(xml_to_dict(child))
        else:
            result[tag] = xml_to_dict(child)
    return result

# Extract relevant data from the SOAP body
namespaces = {'soap': 'http://schemas.xmlsoap.org/soap/envelope/'}
body = root.find('.//soap:Body', namespaces)
response_data = xml_to_dict(body)

# Convert the dictionary to JSON
json_data = json.dumps(response_data, indent=2)

# Display the JSON output
print(json_data)

# Check if the request was successful
if response.status_code == 200:
    print("Request successful.")
    print("Response:")
    print(response.text)
else:
    print(f"Request failed with status code: {response.status_code}")
    print("Response:")
    print(response.text)

In [ ]:
# get a specific employee

# get employees
employee_code: str = '!(+'

# implement the policy transaction endpoint
import requests

# AMS360 SOAP API endpoint
url = "https://affwsapi.ams360.com/v2/service.asmx"

# SOAPAction header
headers = {
    "Content-Type": "text/xml; charset=utf-8",
    "SOAPAction": "http://www.WSAPI.AMS360.com/v2.0/GetEmployee"
}

# SOAP envelope with request details
soap_body = f"""<?xml version="1.0" encoding="utf-8"?>
<soap:Envelope xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:xsd="http://www.w3.org/2001/XMLSchema" xmlns:soap="http://schemas.xmlsoap.org/soap/envelope/">
  <soap:Header>
    <WSAPIAuthToken xmlns="http://www.WSAPI.AMS360.com/v2.0">
      <Token>string</Token>
    </WSAPIAuthToken>
  </soap:Header>
  <soap:Body>
    <GetEmployee_Request xmlns="http://www.WSAPI.AMS360.com/v2.0">
      <EmployeeCode>{employee_code}</EmployeeCode>
      <ShortName></ShortName>
    </GetEmployee_Request>
  </soap:Body>
</soap:Envelope>"""

# Send the POST request
response = requests.post(url, data=soap_body, headers=headers)
print(response.content)

# Parse the XML
root = ET.fromstring(response.content)

# Helper function to convert XML element to dictionary
def xml_to_dict(element):
    if len(element) == 0:  # If no children, return text
        return element.text
    result = {}
    for child in element:
        tag = child.tag.split("}")[-1]  # Remove namespace
        if tag in result:
            if not isinstance(result[tag], list):
                result[tag] = [result[tag]]
            result[tag].append(xml_to_dict(child))
        else:
            result[tag] = xml_to_dict(child)
    return result

# Extract relevant data from the SOAP body
namespaces = {'soap': 'http://schemas.xmlsoap.org/soap/envelope/'}
body = root.find('.//soap:Body', namespaces)
response_data = xml_to_dict(body)

# Convert the dictionary to JSON
json_data = json.dumps(response_data, indent=2)

# Display the JSON output
print(json_data)

# Check if the request was successful
if response.status_code == 200:
    print("Request successful.")
    print("Response:")
    print(response.text)
else:
    print(f"Request failed with status code: {response.status_code}")
    print("Response:")
    print(response.text)

## Testing

In [ ]:
def get_bank_list(session_token: str) -> List[Dict[str, str]]:
    """
    Retrieves a list of all banks from the AMS360 API.
    Args:
        session_token (str): Authentication token for the API
    Returns:
        List[Dict[str, str]]: List of dictionaries containing bank information
    """
    body = """
<BankGetListAll xmlns="http://www.WSAPI.AMS360.com/v3.0">
    <Request xmlns:d4p1="http://www.WSAPI.AMS360.com/v3.0/DataContract" xmlns:i="http://www.w3.org/2001/XMLSchema-instance">
    </Request>
</BankGetListAll>
    """

    root = send_soap_request(session_token, "BankGetListAll", body)
    bank_list = root.findall(".//{http://www.WSAPI.AMS360.com/v3.0/DataContract}BankInfo")

    return [extract_data(bank) for bank in bank_list]

get_bank_list(ticket)

In [ ]:
# get the list of customers
def get_customer_list(session_token: str, name_prefix: str = '%') -> list:
    body = f"""
<CustomerGetListByNamePrefix xmlns="http://www.WSAPI.AMS360.com/v3.0">
    <Request xmlns:d4p1="http://www.WSAPI.AMS360.com/v3.0/DataContract" xmlns:i="http://www.w3.org/2001/XMLSchema-instance">
        <d4p1:NamePrefix>{name_prefix}</d4p1:NamePrefix>
        <d4p1:CustomerType></d4p1:CustomerType>
        <d4p1:IsBrokersCustomer>false</d4p1:IsBrokersCustomer>
    </Request>
</CustomerGetListByNamePrefix>
    """
    root = send_soap_request(session_token, "CustomerGetListByNamePrefix", body)
    customers = root.findall(".//{http://www.WSAPI.AMS360.com/v3.0/DataContract}CustomerInfo")
    return [extract_data(customer) for customer in customers]

get_customer_list(ticket)

In [ ]:
def get_all_customers(session_token: str) -> list:
    """
    Fetches the complete list of customers from the ArrayOfCustomerInfo endpoint.
    """
    # Define the SOAP request body without any specific filters
    body = """
<ArrayOfCustomerInfo xmlns="http://www.WSAPI.AMS360.com/v3.0">
    <Request xmlns:d4p1="http://www.WSAPI.AMS360.com/v3.0/DataContract" xmlns:i="http://www.w3.org/2001/XMLSchema-instance">
    </Request>
</ArrayOfCustomerInfo>
    """

    # Send the SOAP request
    root = send_soap_request(session_token, "ArrayOfCustomerInfo", body)

    # Print the raw XML response for debugging purposes
    print("Raw XML Response:", ET.tostring(root, encoding="unicode"))

    # Attempt to parse CustomerInfo elements with namespace
    namespace = "http://www.WSAPI.AMS360.com/v3.0/DataContract"
    customers = root.findall(f".//{{{namespace}}}CustomerInfo")

    # If no customers are found, it could be due to an unexpected XML structure
    if not customers:
        print("No CustomerInfo elements found in the XML. Please verify the endpoint and namespace.")

    # Extract relevant information from each CustomerInfo element
    customer_list = []
    for customer in customers:
        customer_data = {
            "CustomerId": customer.findtext(f"{{{namespace}}}CustomerId"),
            "CustomerNumber": customer.findtext(f"{{{namespace}}}CustomerNumber"),
            "FirstName": customer.findtext(f"{{{namespace}}}FirstName"),
            "LastName": customer.findtext(f"{{{namespace}}}LastName"),
            "FirmName": customer.findtext(f"{{{namespace}}}FirmName")
        }
        customer_list.append(customer_data)

    return customer_list

get_all_customers(ticket)